In [1]:
from sklearn.ensemble import ExtraTreesRegressor
import pandas as pd

df_processed_with_labels = pd.read_excel('/home/abdeldjalil-hani/Desktop/emergency-sorting-system/data-sets/3. Classified Preprocessed DATA.xlsx')

# Drop the target if it exists
X = df_processed_with_labels.drop(columns=['Emergency_Level'])

# Create dummy target: a random continuous target to extract feature importances
import numpy as np
dummy_target = np.random.rand(len(X))

# Fit model
model = ExtraTreesRegressor(n_estimators=100, random_state=42)
model.fit(X.drop(columns=['Cluster']), dummy_target)

# Get feature importances
importances = model.feature_importances_
feature_weights = dict(zip(X.columns.drop('Cluster'), importances))

In [2]:
feature_weights

{'age': np.float64(0.06461038614459805),
 'gender': np.float64(0.028532357896472016),
 'chest pain type': np.float64(0.013095100657458942),
 'cholesterol': np.float64(0.06212818763435424),
 'exercise angina': np.float64(0.007624460209358483),
 'plasma glucose': np.float64(0.06606510812874233),
 'skin_thickness': np.float64(0.06467013192121489),
 'bmi': np.float64(0.062350363843114584),
 'hypertension': np.float64(0.006002807399567767),
 'heart_disease': np.float64(0.01026206152235881),
 'Residence_type': np.float64(0.026965067413678146),
 'smoking_status': np.float64(0.02839477010697157),
 'Symptom': np.float64(0.05579055257344132),
 'Temperature (¬∞C)': np.float64(0.062923701783153),
 'Heart Rate (bpm)': np.float64(0.05982621848693192),
 'Respiratory Rate (breaths/min)': np.float64(0.05762762575607397),
 'SpO2 (%)': np.float64(0.05916863562279997),
 'Glasgow Score': np.float64(0.04438945872956092),
 'Consciousness': np.float64(0.0034450491967101637),
 'Massive Bleeding': np.float64(0.

In [3]:
from sklearn.preprocessing import MinMaxScaler

scored_df = df_processed_with_labels.copy()
features = [col for col in scored_df.columns if col not in ['Cluster', 'Emergency_Level']]

# Normalize within each cluster
for cluster in scored_df['Emergency_Level'].unique():
    cluster_mask = scored_df['Emergency_Level'] == cluster
    scaler = MinMaxScaler()
    scored_df.loc[cluster_mask, features] = scaler.fit_transform(scored_df.loc[cluster_mask, features])

# Apply weights and compute weighted score
def compute_weighted_score(row):
    return sum(row[feature] * feature_weights[feature] for feature in features)

scored_df['Raw_Score'] = scored_df.apply(compute_weighted_score, axis=1)

/tmp/ipykernel_11912/1106274396.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.24489796 0.18367347 0.2244898  0.34693878 0.40816327 0.42857143
 0.53061224 0.65306122 0.16326531 0.30612245 0.32653061 0.51020408
 0.48979592 0.51020408 0.51020408 0.08163265 0.75510204 0.26530612
 0.53061224 0.53061224 0.14285714 0.48979592 0.63265306 0.16326531
 0.26530612 0.26530612 0.06122449 0.53061224 0.48979592 0.42857143
 0.30612245 0.36734694 0.18367347 0.34693878 0.48979592 0.32653061
 0.59183673 0.55102041 0.08163265 0.55102041 0.53061224 0.30612245
 0.55102041 0.2244898  0.40816327 0.6122449  0.26530612 0.75510204
 0.24489796 0.24489796 0.59183673 0.40816327 0.59183673 0.3877551
 0.20408163 0.20408163 0.63265306 0.3877551  0.48979592 0.6122449
 0.53061224 0.53061224 0.20408163 0.57142857 0.67346939 0.42857143
 0.2244898  0.48979592 0.44897959 0.3877551  0.51020408 0.57142857
 0.2244898  0.30612245 0.53061

In [4]:
for cluster in scored_df['Emergency_Level'].unique():
    cluster_mask = scored_df['Emergency_Level'] == cluster
    raw_scores = scored_df.loc[cluster_mask, 'Raw_Score']
    scaled_scores = MinMaxScaler(feature_range=(1, 99)).fit_transform(raw_scores.values.reshape(-1, 1)).flatten()
    scored_df.loc[cluster_mask, 'Score'] = scaled_scores.round(0).astype(int)

scored_df = scored_df.drop(columns=['Raw_Score', 'Cluster'])
scored_df.to_excel("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/data-sets/4. Scored Classified Preprocessed DATA.xlsx", index=False)

In [5]:
scored_df

,age,gender,chest pain type,cholesterol,exercise angina,plasma glucose,skin_thickness,bmi,hypertension,heart_disease,...,SpO2 (%),Glasgow Score,Consciousness,Massive Bleeding,Respiratory Distress,Risk Factors,blood_pressure,heart_pressure,Emergency_Level,Score
0,0.244898,1,0.333333,1.000000,0,0.703704,0.285714,0.200000,0,0,...,0.966667,0.916667,0.0,0,1,0.800000,0.464912,0.440000,Urgent (L3),58.0
1,0.947368,0,0.750000,0.272727,0,0.201448,0.337662,0.269710,0,0,...,0.571429,0.500000,0.5,0,0,0.666667,0.191667,0.225000,Non-Urgent (L5),30.0
2,0.183673,1,0.333333,1.000000,0,0.518519,0.415584,1.000000,0,0,...,0.833333,0.833333,0.0,0,1,0.200000,0.526316,0.493333,Urgent (L3),56.0
3,0.894737,0,1.000000,0.581818,1,0.169506,0.389610,0.269710,0,0,...,0.642857,0.583333,0.5,0,0,0.000000,0.550000,0.387500,Non-Urgent (L5),50.0
4,0.471698,1,0.750000,0.562500,0,0.683734,0.809524,0.189381,0,0,...,0.933333,0.666667,0.0,0,0,0.000000,0.517857,0.577778,Emergent (L2),66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6957,0.962264,0,0.000000,0.038961,0,0.397464,0.250000,0.125000,1,0,...,0.666667,0.666667,0.0,0,1,0.800000,0.696970,0.687500,Resuscitation (L1),50.0
6958,0.981132,0,0.000000,0.090909,0,0.974923,0.050000,0.530303,0,0,...,0.666667,1.000000,0.0,0,0,0.800000,0.378788,0.250000,Resuscitation (L1),61.0
6959,0.981132,0,0.000000,0.454545,0,0.386877,0.500000,0.352273,0,0,...,0.866667,0.666667,0.0,0,0,0.066667,0.500000,0.375000,Resuscitation (L1),54.0
6960,0.261905,1,0.000000,0.177419,0,0.589380,0.166667,0.266112,0,0,...,0.625000,0.750000,0.0,0,1,0.766667,0.100000,0.550000,Less Urgent (L4),43.0
